In [ ]:
import numpy as np

from barcode import Barcode
from cell_state import CellTypeTree
from clt_simulator import CLTSimulator
from barcode_simulator import BarcodeSimulator
from alignment import AlignerNW
from clt_observer import CLTObserver
from IPython.display import display
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
n_trees = 10
time = 3
error_rate = .01
target_lambdas = np.array([0.1 for _ in range(10)])
repair_lambdas = np.array([1 for _ in range(10)])
indel_probability = .5
left_deletion_lambda = 3
right_deletion_lambda = 3
insertion_lambda = .2
birth_lambda = 1.25
death_lambda = 0.01

In [ ]:
# Create a cell-type tree
cell_types = ["brain", "eye"]
cell_type_tree = CellTypeTree(cell_type=None, rate=0.1, probability=1.0)
cell_type_tree.add_child(
    CellTypeTree(cell_type=0, rate=0, probability=0.5))
cell_type_tree.add_child(
    CellTypeTree(cell_type=1, rate=0, probability=0.5))

In [ ]:
# Instantiate barcode and tree simulators, and observer
bcode_simulator = BarcodeSimulator(np.array(target_lambdas),
                                   np.array(repair_lambdas),
                                   indel_probability,
                                   left_deletion_lambda,
                                   right_deletion_lambda,
                                   insertion_lambda)
clt_simulator = CLTSimulator(birth_lambda, death_lambda,cell_type_tree, bcode_simulator)
observer = CLTObserver(sampling_rate=1, error_rate=error_rate)

In [ ]:
# Simulate a forest of observed leaves and pruned trees
forest_obs_leaves, forest_pruned_clt = zip(*sorted((observer.observe_leaves(clt_simulator.simulate(time)) for _ in range(n_trees)), key=lambda x: len(x[0])))
# display the first of n
display(forest_pruned_clt[0].savefig("%%inline"))

In [ ]:
def alignment_experiment(params_list):
    """take a list of alignment parameter dictionaries and make a data frame of event calling results"""
    df = pd.DataFrame(data=[[sim,
                             sum(leaf.barcode.get_events() == leaf.barcode.get_events(aligner=AlignerNW(**params)) for leaf in obs_leaves),
                             len(obs_leaves), str(params)]
                            for sim, obs_leaves in enumerate(forest_obs_leaves, 1) for params in params_list],
                      columns=('simulation', 'correctly annotated barcodes', 'simulated barcodes', 'NW parameters'))
    plt.figure(figsize=(10,5))
    sns.pointplot(y='simulated barcodes', x='simulation', data=df,
                  clip_on=False, color='gray', linestyles='--')    
    sns.barplot(y='correctly annotated barcodes', x='simulation', hue='NW parameters', data=df)
    plt.tight_layout()
    plt.show()
#     sns.lmplot(x='simulated barcodes', y='correctly annotated barcodes', hue='NW parameters', data=df,
#                fit_reg=False, scatter_kws=dict(alpha=.4, clip_on=False))
#     ax_max = max(df['simulated barcodes'])
#     plt.xlim([0, ax_max])
#     plt.ylim([0, ax_max])
#     plt.plot([0, ax_max], [0, ax_max], color='gray', ls='--')
#     plt.show()

In [ ]:
"""
an experiment with different gap open penalties
"""
params_list = [dict(mismatch=-1, gap_open=gap_open, gap_extend=-.5) for gap_open in (-15, -10, -5, -.5)]
alignment_experiment(params_list)

In [ ]:
"""
an experiment with different gap extension penalties
"""
params_list = [dict(mismatch=-1, gap_open=-10, gap_extend=gap_extend) for gap_extend in (-3, -2, -1, -.5)]
alignment_experiment(params_list)

In [ ]:
"""
an experiment with different mismatch penalties
"""
params_list = [dict(mismatch=mismatch, gap_open=-10, gap_extend=-0.5) for mismatch in (-3, -2, -1, -.5)]
alignment_experiment(params_list)